# Лабораторна робота № 5

## ІП-14 Бабіч Денис

---

# Підготовчий етап

## Імпортування необхідних модулів

In [1]:
import cv2
import logging
import warnings
import numpy as np
from enum import Enum
import tensorflow as tf
from imageai.Detection import ObjectDetection, VideoObjectDetection

## Допоміжні функції

In [2]:
class InputType(Enum):
    IMAGE = 0
    VIDEO = 1

def initialize_detection_model(model_path: str, input_type: InputType) -> ObjectDetection:
    tf.get_logger().setLevel(logging.ERROR)
    warnings.filterwarnings("ignore", category = UserWarning, message = ".*Argument `alpha` is deprecated.*")

    if (input_type == InputType.IMAGE):
        model = ObjectDetection()
    else:
        model = VideoObjectDetection()

    model.setModelTypeAsTinyYOLOv3()
    model.setModelPath(model_path)
    model.loadModel()

    return model

def show_image(image: np.array, title: str = "Preview") -> None:
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def overlay_detected_objects(frame_number: int, detected_objects: np.array, objects_count: int, frame: np.array):
    OBJECT_NAME_OFFSET = 5
    FLOATING_PRECISION = 2
    BOX_OUTILNE_THICKNESS = 2
    BOX_OUTILNE_COLOR = (0, 255, 0)

    for detected_object in detected_objects:
        name = detected_object["name"]
        x1, y1, x2, y2 = detected_object["box_points"]
        percentage_probability = round(detected_object["percentage_probability"], FLOATING_PRECISION)

        cv2.rectangle(frame, (x1, y1), (x2, y2), BOX_OUTILNE_COLOR, BOX_OUTILNE_THICKNESS)
        cv2.putText(frame, f"{name} : {percentage_probability}%", (x1, y1 - OBJECT_NAME_OFFSET), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_OUTILNE_COLOR, BOX_OUTILNE_THICKNESS)

    return frame

---

# Основний етап

## Застосування на прикладі зображення

In [3]:
PATH_INPUT = "input/image.jpg"
PATH_OUTPUT = "output/image.jpg"
PATH_MODEL = "models/yolo-tiny.h5"

image = cv2.imread(PATH_INPUT)
model = initialize_detection_model(PATH_MODEL, InputType.IMAGE)

show_image(image, "Input Image")

_, detected_objects = model.detectObjectsFromImage(input_image = image, 
                                                   input_type = "array", 
                                                   output_type = "array", 
                                                   display_box = False, 
                                                   display_object_name = False, 
                                                   display_percentage_probability = False)

overlay_detected_objects(0, detected_objects, 0, image)

show_image(image, "Output Image")

_ = cv2.imwrite(PATH_OUTPUT, image)

## Застосування на прикладі відео

In [4]:
PATH_OUTPUT = "output/"
PATH_INPUT = "input/video.mp4"
PATH_MODEL = "models/yolo-tiny.h5"

MIN_PERCENTAGE_PROBABILITY = 30
FPS = int(cv2.VideoCapture("input/video.mp4").get(cv2.CAP_PROP_FPS))

model = initialize_detection_model(PATH_MODEL, InputType.VIDEO)

video_path = model.detectObjectsFromVideo(
    input_file_path = PATH_INPUT,
    output_file_path = PATH_OUTPUT,
    frames_per_second = FPS,
    per_frame_function = overlay_detected_objects,
    return_detected_frame = True,
    minimum_percentage_probability = MIN_PERCENTAGE_PROBABILITY,
    save_detected_video = True
)

print("Done!")

Done!
